In [3]:
# BATCH PROCESSING 
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.fft import fft2, fftshift, ifft2, fft, fftfreq
from scipy.signal import find_peaks, peak_widths

# === SETTINGS ===
image_folder = "W:\\Users\\Daphne\\WT_RESULTS\\WT_ExM_SR\\Maximum projections\ROIs for analysis"
pixel_size_um = 0.288  # microns per pixel

# === OUTPUT SETUP ===
output_folder = os.path.join(image_folder, "output")
os.makedirs(output_folder, exist_ok=True)

results = []  # List to store measurement results

# === PROCESS EACH IMAGE IN FOLDER ===  
# WORKS BEST WITH PNG IMAGES!
for filename in os.listdir(image_folder):
    if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tif', '.tiff')):
        continue  # Skip non-image files

    image_path = os.path.join(image_folder, filename)
    print(f"Processing: {filename}")

    # --- Load image in grayscale ---
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Could not load {filename}. Skipping.")
        continue

    h, w = img.shape

    # === FFT ANALYSIS ===
    f = fft2(img)
    fshift = fftshift(f)
    power_spectrum = np.abs(fshift)**2

    # 1. FFT along vertical axis (Y-direction)
    fft_y = fft(img, axis=0)
    power_y = np.abs(fft_y)**2 # square of the magnitude of the FFT gives the power spectrum --> helps to identify dominant frequencies

    # 2. Mean across image columns
    mean_power_y = fftshift(power_y.mean(axis=1))
    freqs_y = fftshift(fftfreq(h, d=pixel_size_um))  # Frequencies in 1/um

    # 3. Use only positive frequencies
    pos_mask = freqs_y > 0
    pos_freqs = freqs_y[pos_mask]
    pos_power = mean_power_y[pos_mask]

    # 4. Find frequency peaks
    peaks, _ = find_peaks(pos_power, distance=10, prominence=10^7)  # Adjust prominence and distance as needed
    if len(peaks) > 0:
        
        # # METHOD 1: Find the first peak 
        # first_peak_freq = pos_freqs[peaks[0]]
        # spacing_um_fft = 1 / first_peak_freq  # Convert freq to spacing

        # METHOD 2: Find the index of the peak with the maximum power aka the most dominant frequency
        # This gives the most significant frequency component in the image, which corresponds to the dominant spacing
        dominant_peak_idx = peaks[np.argmax(pos_power[peaks])]
        dominant_peak_freq = pos_freqs[dominant_peak_idx]
        spacing_um_fft = 1 / dominant_peak_freq

        # # METHOD 3: peak must be within a certain range
        # # Define frequency range (e.g. expecting spacing between 5 and 10 µm → freq between 0.1 and 0.2 µm⁻¹)
        # min_freq = 1 / 10    # 0.1 µm⁻¹
        # max_freq = 1 / 5    # 0.2 µm⁻¹
        # # Mask to keep peaks in desired frequency range
        # valid_peaks = [p for p in peaks if min_freq <= pos_freqs[p] <= max_freq]
        # if valid_peaks:
        #     # Pick the strongest peak within range
        #     best_peak_idx = max(valid_peaks, key=lambda p: pos_power[p])
        #     peak_freq = pos_freqs[best_peak_idx]
        #     spacing_um_fft = 1 / peak_freq
        #     # Optional: Estimate error
        #     results = peak_widths(pos_power, [best_peak_idx], rel_height=0.5)
        #     width_freq = results[0][0]
        #     spacing_um_fft_err = spacing_um_fft * (width_freq / best_peak_idx)
        # else:
        #     spacing_um_fft = spacing_um_fft_err = None

        width_freq = peak_widths(pos_power, peaks, rel_height=0.5)[0][0]
        spacing_um_fft_err = spacing_um_fft * (width_freq / peaks[0])
    else:
        spacing_um_fft = spacing_um_fft_err = None

    # === AUTOCORRELATION ANALYSIS ===
    # The power spectrum is equivalent to squaring the magnitude of the Fourier transform. 
    # We take the inverse 2D FFT (ifft2) of the power spectrum to get the autocorrelation function.
    # This gives a 2D autocorrelation image that shows how the original image correlates with itself at different spatial shifts in both X and Y directions.
    power_spec = np.abs(f)**2
    autocorr = fftshift(ifft2(power_spec).real) # .real removes imaginary part (artifacts)
    autocorr /= autocorr.max()  # Normalize so tha max value is 1 to simplify peak detection

    # Project along columns (i.e., vertical autocorrelation)
    # Since we are interested in vertical spacing (due to horizontal lines), we average the autocorrelation image across columns resulting in a 1D profile along Y (vertical axis).
    # This gives a signal representing how correlated each vertical shift is. (autocorrelation along Y axis)
    autocorr_y = autocorr.mean(axis=1) 
    lags = np.arange(-h//2, h//2) * pixel_size_um  # This gives the actual shift in microns (lags) corresponding to each point in the autocorrelation curve. Ranges from -height/2 to +height/2 in steps of 1 pixel × pixel size.
    center_idx = len(lags) // 2
    lags_positive = lags[center_idx:] # Since the autocorrelation function is symmetric, we only analyze the positive side (i.e., positive shifts).
    autocorr_y_positive = autocorr_y[center_idx:]

    # Find autocorrelation peaks
    # peaks in the 1D autocorrelation curve that represent repeating patterns. The first prominent peak (after zero lag) corresponds to the first repeat distance = vertical spacing between horizontal lines.
    # The position of the first peak gives the estimated vertical spacing in microns. The FWHM (full width at half maximum) of the peak gives an error estimate, based on how sharp that peak is
    peaks_auto, _ = find_peaks(autocorr_y_positive, height=0.2, distance=20) 
    if len(peaks_auto) > 0:
        spacing_um_auto = lags_positive[peaks_auto[0]]
        width_lag = peak_widths(autocorr_y_positive, peaks_auto, rel_height=0.5)[0][0]
        spacing_um_auto_err = spacing_um_auto * (width_lag / peaks_auto[0])
    else:
        spacing_um_auto = spacing_um_auto_err = None

    # === PLOTTING ===
    plt.figure(figsize=(14, 4))

    # FFT Spectrum
    plt.subplot(1, 3, 1)
    plt.imshow(np.log1p(np.abs(fshift)), cmap='gray')
    plt.title(f"{filename}\nFFT Spectrum (log scale)")

    # Power Spectrum Plot
    plt.subplot(1, 3, 2)
    plt.plot(pos_freqs, pos_power)
    if len(peaks) > 0:
        plt.plot(pos_freqs[peaks], pos_power[peaks], "rx")
    plt.title("Mean Power Spectrum (Y axis)")
    plt.xlabel("Spatial Frequency (1/µm)")
    plt.ylabel("Power")

    # Autocorrelation Plot
    plt.subplot(1, 3, 3)
    plt.plot(lags_positive, autocorr_y_positive)
    if len(peaks_auto) > 0:
        plt.plot(lags_positive[peaks_auto], autocorr_y_positive[peaks_auto], "rx")
    plt.title("Autocorrelation (Y axis)")
    plt.xlabel("Lag (µm)")
    plt.ylabel("Correlation")

    plt.tight_layout()
    plot_filename = os.path.splitext(filename)[0] + "_analysis.png"
    plt.savefig(os.path.join(output_folder, plot_filename), dpi=200)
    plt.close()

    # === SAVE RESULTS ===
    results.append({
        "Filename": filename,
        "FFT_Spacing_um": spacing_um_fft,
        "FFT_Spacing_Error_um": spacing_um_fft_err,
        "Autocorr_Spacing_um": spacing_um_auto,
        "Autocorr_Spacing_Error_um": spacing_um_auto_err
    })

# === WRITE RESULTS TO CSV ===
results_df = pd.DataFrame(results)
csv_path = os.path.join(output_folder, "spacing_results.csv")
results_df.to_csv(csv_path, index=False)

print(f"\nAll results saved to: {csv_path}")


Processing: MAX_ExM_ptetWT_SR_cell4_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell5_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell24_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell15_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell14_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell19_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell19_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell21_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell3_dorsal_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell18_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell25_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell27_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell6_dorsal_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell22_ventral_singlets.png
Processing: MAX_ExM_ptetWT_SR_cell2_dorsal2_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell22_ventral_doublets.png
Processing: MAX_ExM_ptetWT_SR_cell2_dorsal_singlets.png
Processing: MAX_ExM_ptetWT

In [ ]:
# FOR ONLY ONE IMAGE, TO TEST
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, fftshift, ifft2, fft, fftfreq
from scipy.signal import find_peaks, peak_widths

# === SETTINGS ===
image_path = "W:\\Users\\Daphne\\WT_RESULTS\\WT_ExM_SR\\Maximum projections\\ROIs for analysis\\MAX_ExM_ptetWT_SR_cell1_dorsal_singlets_binaryridges-5.png"
pixel_size_um = 0.288  # microns per pixel

# === LOAD IMAGE ===
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"Could not load image at: {image_path}")
h, w = img.shape

# === FFT ANALYSIS ===
f = fft2(img)
fshift = fftshift(f)
power_spectrum = np.abs(fshift)**2

# 1. FFT along vertical axis
fft_y = fft(img, axis=0)
power_y = np.abs(fft_y)**2

# 2. Mean across columns and shift
mean_power_y = fftshift(power_y.mean(axis=1))
freqs_y = fftshift(fftfreq(h, d=pixel_size_um))

# 3. Use only positive frequencies
pos_mask = freqs_y > 0
pos_freqs = freqs_y[pos_mask]
pos_power = mean_power_y[pos_mask]

# 4. Find peaks on the aligned data
peaks, _ = find_peaks(pos_power, distance=5, prominence=0.1)

if len(peaks) > 0:
    first_peak_freq = freqs_y[pos_mask][peaks[0]]
    spacing_um_fft = 1 / first_peak_freq

    results = peak_widths(mean_power_y[pos_mask], peaks, rel_height=0.5)
    width_freq = results[0][0]
    spacing_um_fft_err = spacing_um_fft * (width_freq / peaks[0])
else:
    spacing_um_fft = spacing_um_fft_err = None

# === AUTOCORRELATION ANALYSIS ===
power_spec = np.abs(f)**2
autocorr = fftshift(ifft2(power_spec).real)
autocorr /= autocorr.max()

# Analyze Y-direction autocorrelation
autocorr_y = autocorr.mean(axis=1)
lags = np.arange(-h//2, h//2) * pixel_size_um
center_idx = len(lags) // 2
lags_positive = lags[center_idx:]
autocorr_y_positive = autocorr_y[center_idx:]

peaks_auto, _ = find_peaks(autocorr_y_positive, height=0.1, distance=5)
if len(peaks_auto) > 0:
    spacing_um_auto = lags_positive[peaks_auto[0]]
    results_auto = peak_widths(autocorr_y_positive, peaks_auto, rel_height=0.5)
    width_lag = results_auto[0][0]
    spacing_um_auto_err = spacing_um_auto * (width_lag / peaks_auto[0])
else:
    spacing_um_auto = spacing_um_auto_err = None

# === PLOTTING ===
plt.figure(figsize=(14, 4))
plt.subplot(1, 3, 1)
plt.imshow(np.log1p(np.abs(fshift)), cmap='gray')
plt.title("FFT Spectrum (log scale)")

plt.subplot(1, 3, 2)
plt.plot(pos_freqs, pos_power)
plt.plot(pos_freqs[peaks], pos_power[peaks], "rx")
plt.title("Mean Power Spectrum (Y direction)")
plt.xlabel("Spatial Frequency (1/µm)")
plt.ylabel("Power")

plt.subplot(1, 3, 3)
plt.plot(lags_positive, autocorr_y_positive)
plt.plot(lags_positive[peaks_auto], autocorr_y_positive[peaks_auto], "rx")
plt.title("Autocorrelation Profile (Y direction)")
plt.xlabel("Lag (µm)")
plt.ylabel("Correlation")

plt.tight_layout()
plt.show()

# === RESULTS ===
print("\n=== ESTIMATED SPACING ===")
if spacing_um_fft:
    print(f"FFT Estimate: {spacing_um_fft:.2f} µm ± {spacing_um_fft_err:.2f} µm (FWHM)")
else:
    print("FFT Estimate: No clear peak found.")

if spacing_um_auto:
    print(f"Autocorrelation Estimate: {spacing_um_auto:.2f} µm ± {spacing_um_auto_err:.2f} µm (FWHM)")
else:
    print("Autocorrelation Estimate: No clear peak found.")


In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.signal import find_peaks, peak_widths
# from scipy.fft import fft2, fftshift, ifft2, fft

# # === SETTINGS ===
# # image_path = "W:\\Users\\Daphne\\Imaging_Daphne\\25-06-02_XC_ptetwt_ExM\\MAX_020625_ExM_ptetWT_20xw_NSpark_288x0.589_002-ventral-crop-2rotated.png"
# image_path = "W:\\Users\\Daphne\\WT_RESULTS\\WT_ExM_SR\\Maximum projections\\ROIs for analysis\\MAX_ExM_ptetWT_SR_cell1_dorsal_singlets_binaryridges-4.png"
# pixel_size_um = 0.288  # micrometers per pixel

# # === LOAD IMAGE ===
# img_orig = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
# img = img_orig.copy()

# # # === FFT METHOD & RADIAL PROFILE ===  OLD!!! Doesn't include the dominant frequency orientation so len(radial_profile) parameter is not giving the correct spacing
# # f = np.fft.fft2(img_orig)
# # fshift = np.fft.fftshift(f)
# # magnitude = np.abs(fshift)
# # magnitude_log = np.log(1 + magnitude)

# center = np.array(magnitude.shape) // 2
# Y, X = np.indices(magnitude.shape)
# r = np.hypot(X - center[1], Y - center[0]).astype(int)
# radial_profile = np.bincount(r.ravel(), magnitude.ravel()) / np.bincount(r.ravel())
# peaks_fft, _ = find_peaks(radial_profile, height=radial_profile.max()*0.001, distance=5)

# # if len(peaks_fft) > 0:
# #     freq_px = peaks_fft[0] # In frequency space, the first peak corresponds to the dominant frequency, this line assumes the first peak is the dominant one and choses it as the frequency index
# #     spacing_px_fft = len(radial_profile) / freq_px 
# #     spacing_um_fft = spacing_px_fft * pixel_size_um
# #     # Error estimate: width at half max
# #     results_fft = peak_widths(radial_profile, peaks_fft, rel_height=0.05)
# #     width_fft_px = results_fft[0][0]
# #     # Convert width in frequency index to spacing error in microns
# #     # The error in spacing is proportional to the width of the peak in frequency space
# #     spacing_um_fft_err = spacing_um_fft * (width_fft_px / freq_px)
# # else:
# #     spacing_px_fft = spacing_um_fft = width_fft_px = spacing_um_fft_err = None

# # === FFT METHOD WITH AUTO-ORIENTATION DETECTION ===
# f = np.fft.fft2(img_orig)
# fshift = np.fft.fftshift(f)
# magnitude = np.abs(fshift)
# magnitude_log = np.log1p(magnitude)

# # Normalize and threshold to find bright spots (excluding center)
# normalized = cv2.normalize(magnitude_log, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
# threshold_val = 20
# _, binary_thresh = cv2.threshold(normalized, threshold_val, 255, cv2.THRESH_BINARY)

# # Mask out small radius around center to ignore DC
# mask = np.ones_like(binary_thresh, dtype=np.uint8) * 255
# cv2.circle(mask, tuple(center), radius=5, color=0, thickness=-1)
# binary_thresh = cv2.bitwise_and(binary_thresh, mask)

# # Find contours (bright FFT peaks)
# contours, _ = cv2.findContours(binary_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# angles = []

# for cnt in contours:
#     M = cv2.moments(cnt)
#     if M["m00"] != 0:
#         cx = int(M["m10"] / M["m00"])
#         cy = int(M["m01"] / M["m00"])
#         dx = cx - center[1]
#         dy = cy - center[0]
#         angle = np.arctan2(dy, dx) * 180 / np.pi
#         angles.append(angle)

# if angles:
#     # Use the most common orientation rounded to nearest 10 degrees
#     dominant_angle = np.median(angles) + 90 
#     print(f"Detected dominant FFT orientation: {dominant_angle:.2f} degrees")

#     # Convert angle to radians and get directional slice
#     angle_rad = np.deg2rad(dominant_angle)
#     # Sample line through FFT spectrum along that angle
#     from skimage.measure import profile_line

#     line_length = min(magnitude.shape) // 2
#     x0 = center[1] - int(np.cos(angle_rad) * line_length)
#     y0 = center[0] - int(np.sin(angle_rad) * line_length)
#     x1 = center[1] + int(np.cos(angle_rad) * line_length)
#     y1 = center[0] + int(np.sin(angle_rad) * line_length)
#     directional_profile = profile_line(magnitude, (y0, x0), (y1, x1))

#     # Detect peaks in directional FFT
#     directional_profile = directional_profile - directional_profile.min()
#     directional_profile /= directional_profile.max()
#     peaks_dir, _ = find_peaks(directional_profile, height=0.05, distance=5)

#     if len(peaks_dir) > 0:
#         freq_idx = peaks_dir[0]
#         spacing_px_dir = (2 * line_length) / freq_idx
#         spacing_um_dir = spacing_px_dir * pixel_size_um

#         # Estimate error via FWHM
#         results_dir = peak_widths(directional_profile, peaks_dir, rel_height=0.5)
#         width_px = results_dir[0][0]
#         spacing_um_dir_err = spacing_um_dir * (width_px / freq_idx)
#     else:
#         spacing_um_dir = spacing_um_dir_err = None

#     # === Plot Directional FFT Profile and Peaks ===
#     plt.figure(figsize=(10, 4))
#     plt.plot(directional_profile, label='FFT profile')
#     plt.plot(peaks_dir, directional_profile[peaks_dir], "rx", label='Peaks')
#     plt.title("FFT Profile Along Dominant Direction")
#     plt.xlabel("Position along profile (px)")
#     plt.ylabel("Normalized Intensity")
#     plt.legend()
#     plt.grid(True)
#     plt.show()

#     # === Estimate average spacing from multiple peaks ===
#     if len(peaks_dir) > 1:
#         # Calculate all pairwise distances
#         peak_distances_px = np.diff(peaks_dir)
#         peak_spacings_px = len(directional_profile) / peak_distances_px
#         # peak_spacings_um = peak_distances_px * pixel_size_um
#         peak_spacings_um = peak_spacings_px * pixel_size_um
#         spacing_um_dir_avg = np.mean(peak_spacings_um)
#         spacing_um_dir_std = np.std(peak_spacings_um)

#         print(f"FFT (Avg Directional): {spacing_um_dir_avg:.2f} µm ± {spacing_um_dir_std:.2f} µm (SD)")
#     else:
#         print("FFT (Avg Directional): Only one peak found; cannot estimate average spacing.")

# else:
#     print("Could not detect dominant orientation in FFT.")
#     spacing_um_dir = spacing_um_dir_err = None




# # === FFT IN Y DIRECTION ===
# fft_y = fft(fshift, axis=0)
# # Calculate the power spectrum
# power_spectrum_y = np.abs(fft_y) ** 2
# # Calculate the average power spectrum across all columns
# power_spectrum_y_mean = power_spectrum_y.mean(axis=1)
# # Calculate the frequencies corresponding to the power spectrum
# freqs_y = np.fft.fftfreq(fft_y.shape[0], d=pixel_size_um)
# freqs_y = fftshift(freqs_y)
# # Find the first peak after zero frequency (only positive frequencies)
# pos_mask_y = freqs_y > 0
# peaks_fft_y, _ = find_peaks(power_spectrum_y_mean[pos_mask_y], height=10, distance=5)
# if len(peaks_fft_y) > 0:
#     first_peak_idx_y = peaks_fft_y[0]
#     first_peak_freq_y = freqs_y[pos_mask_y][first_peak_idx_y]
#     spacing_um_fft_y = 1 / first_peak_freq_y # the inverse of the frequency gives the spacing in micrometers
#     # Error estimate: width at half max
#     results_fft_y = peak_widths(power_spectrum_y_mean[pos_mask_y], peaks_fft_y, rel_height=0.5)
#     width_fft_freq_y = results_fft_y[0][0]
#     # Convert width in frequency to error in spacing
#     freq_at_peak_y = first_peak_freq_y
#     spacing_um_fft_y_err = spacing_um_fft_y * (width_fft_freq_y / first_peak_idx_y)
# else:
#     spacing_um_fft_y = width_fft_freq_y = spacing_um_fft_y_err = None


# # === 2D AUTOCORRELATION & PSD IN Y DIRECTION ===
# power_spectrum = np.abs(f) ** 2
# autocorr = fftshift(ifft2(power_spectrum).real)
# autocorr /= np.max(autocorr)

# psd_y = np.abs(fft(autocorr, axis=0))**2
# psd_y_mean = psd_y.mean(axis=1)
# freqs = np.fft.fftfreq(autocorr.shape[0], d=pixel_size_um)
# freqs = fftshift(freqs)
# psd_y_mean = fftshift(psd_y_mean)

# # Find the first peak after zero frequency (only positive frequencies)
# pos_mask = freqs > 0
# peaks_psd, _ = find_peaks(psd_y_mean[pos_mask], height=10, distance=5)
# if len(peaks_psd) > 0:
#     first_peak_idx = peaks_psd[0]
#     first_peak_freq = freqs[pos_mask][first_peak_idx]
#     spacing_um_psd = 1 / first_peak_freq
#     # Error estimate: width at half max
#     results_psd = peak_widths(psd_y_mean[pos_mask], peaks_psd, rel_height=0.5)
#     width_psd_freq = results_psd[0][0]
#     # Convert width in frequency to error in spacing
#     # The error in spacing is proportional to the width of the peak in frequency space
#     freq_at_peak = first_peak_freq
#     spacing_um_psd_err = spacing_um_psd * (width_psd_freq / first_peak_idx)
# else:
#     spacing_um_psd = width_psd_freq = spacing_um_psd_err = None

# # === 2D AUTOCORRELATION & PSD IN X DIRECTION ===
# psd_x = np.abs(fft(autocorr, axis=1))**2
# psd_x_mean = psd_x.mean(axis=0)
# freqs_x = np.fft.fftfreq(autocorr.shape[1], d=pixel_size_um)
# freqs_x = fftshift(freqs_x)
# # Find the first peak after zero frequency (only positive frequencies)
# pos_mask_x = freqs_x > 0
# peaks_psd_x, _ = find_peaks(psd_x_mean[pos_mask_x], height=10, distance=5)
# if len(peaks_psd_x) > 0:
#     first_peak_idx_x = peaks_psd_x[0]
#     first_peak_freq_x = freqs_x[pos_mask_x][first_peak_idx_x]
#     spacing_um_psd_x = 1 / first_peak_freq_x
#     # Error estimate: width at half max
#     results_psd_x = peak_widths(psd_x_mean[pos_mask_x], peaks_psd_x, rel_height=0.5)
#     width_psd_freq_x = results_psd_x[0][0]
#     # Convert width in frequency to error in spacing
#     freq_at_peak_x = first_peak_freq_x
#     spacing_um_psd_err_x = spacing_um_psd_x * (width_psd_freq_x / first_peak_idx_x)
# else:
#     spacing_um_psd_x = width_psd_freq_x = spacing_um_psd_err_x = None


# # === PLOTS ===
# fig, axs = plt.subplots(2, 2, figsize=(16, 8))

# axs[0, 0].imshow(img_orig, cmap='gray')
# axs[0, 0].set_title("Original Image")

# axs[1, 0].imshow(magnitude_log, cmap='gray')
# axs[1, 0].set_title("FFT Spectrum (log)")

# if magnitude_log.dtype != np.uint8:
#     magnitude_log = cv2.normalize(magnitude_log, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
# axs[0, 1].imshow(magnitude_log, cmap='gray', vmin=180, vmax=255)
# axs[0, 1].set_title("FFT Spectrum only showing high intensities")

# axs[1, 1].plot(radial_profile)
# if len(peaks_fft) > 0:
#     axs[1, 1].plot(peaks_fft, radial_profile[peaks_fft], "x")
# axs[1, 1].set_ylim(0, radial_profile.max() * 0.1)
# # axs[1,1].set_xlim(0, 30)
# axs[1, 1].set_xlabel("Frequency Index (px)")
# axs[1, 1].set_ylabel("Radial Profile")
# axs[1, 1].axhline(y=radial_profile.max() * 0.001, color='r', linestyle='--', label='Threshold')
# axs[1, 1].legend()
# axs[1, 1].set_xlim(0, len(radial_profile) // 2)
# axs[1, 1].set_xticks(np.arange(0, len(radial_profile) // 2 + 1, step=10))
# axs[1, 1].set_xticklabels(np.arange(0, len(radial_profile) // 2 + 1, step=10) * pixel_size_um)
# axs[1, 1].set_title("FFT Radial Profile")

# plt.tight_layout()
# plt.show()

# # === VISUALIZE DETECTED ORIENTATION ON FFT ===
# fft_vis = cv2.cvtColor(normalized.copy(), cv2.COLOR_GRAY2BGR)
# if angles:
#     # Convert to int coordinates
#     pt1 = (x0, y0)
#     pt2 = (x1, y1)
#     # Draw the orientation line
#     cv2.line(fft_vis, pt1, pt2, (0, 0, 255), 2)
#     cv2.circle(fft_vis, tuple(center), 5, (0, 255, 0), -1)
#     plt.figure(figsize=(6, 6))
#     plt.imshow(fft_vis)
#     plt.title(f"Dominant Direction in FFT: {dominant_angle:.1f}°")
#     plt.axis('off')
#     plt.show()

# plt.figure(figsize=(8, 5))
# plt.plot(freqs, psd_y_mean)
# plt.plot(freqs[pos_mask][peaks_psd], psd_y_mean[pos_mask][peaks_psd], "rx", label="Peaks")
# plt.legend()
# plt.title("Power Spectral Density (PSD) in Y Direction (Autocorrelation)")
# plt.xlabel("Spatial Frequency (1/µm)")
# plt.ylabel("Power")
# plt.xlim(0, np.max(freqs))
# plt.ylim(0, 1000)
# plt.grid(True)
# plt.show()

# plt.figure(figsize=(8, 5))
# plt.plot(freqs_x, psd_x_mean)
# plt.plot(freqs_x[pos_mask_x][peaks_psd_x], psd_x_mean[pos_mask_x][peaks_psd_x], "rx", label="Peaks")
# plt.legend()
# plt.title("Power Spectral Density (PSD) in X Direction (Autocorrelation)")
# plt.xlabel("Spatial Frequency (1/µm)")
# plt.ylabel("Power")
# plt.xlim(0, np.max(freqs_x))
# plt.ylim(0, 1000)
# plt.grid(True)
# plt.show()

# # === RESULTS ===
# if spacing_um_dir:
#     print(f"FFT (Auto Directional): {spacing_um_dir:.2f} µm ± {spacing_um_dir_err:.2f} µm (FWHM)")
# else:
#     print("FFT (Auto Directional): Could not detect frequency along dominant direction")


# print("\n=== ESTIMATED AVERAGE SPACING AND ERRORS ===")
# if spacing_um_fft:
#     print(f"FFT: {spacing_px_fft:.2f} px ≈ {spacing_um_fft:.2f} µm ± {spacing_um_fft_err:.2f} µm (FWHM)")
# else:
#     print("FFT: Could not detect dominant frequency")

# if spacing_um_psd:
#     print(f"PSD Y Direction (Autocorrelation): {spacing_um_psd:.2f} µm ± {spacing_um_psd_err:.2f} µm (FWHM)")
# else:
#     print("PSD Y Direction: Could not detect dominant frequency")
# if spacing_um_psd_x:
#     print(f"PSD X Direction (Autocorrelation): {spacing_um_psd_x:.2f} µm ± {spacing_um_psd_err_x:.2f} µm (FWHM)")
# else:
#     print("PSD X Direction: Could not detect dominant frequency")

# if spacing_um_fft_y:
#     print(f"FFT Y Direction: {spacing_um_fft_y:.2f} µm ± {spacing_um_fft_y_err:.2f} µm (FWHM)")
# else:
#     print("FFT Y Direction: Could not detect dominant frequency")

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, fftshift, ifft2, fft, fftfreq
from scipy.signal import find_peaks, peak_widths

# === SETTINGS ===
image_path = "W:\\Users\\Daphne\\WT_RESULTS\\WT_ExM_SR\\Maximum projections\\ROIs for analysis\\MAX_ExM_ptetWT_SR_cell1_dorsal_singlets_binaryridges-5.png"
pixel_size_um = 0.288  # microns per pixel

# === LOAD IMAGE ===
img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError(f"Could not load image at: {image_path}")
h, w = img.shape

# === FFT ANALYSIS ===
f = fft2(img)
fshift = fftshift(f)
power_spectrum = np.abs(fshift)**2

# Take mean spectrum in vertical (Y) direction
# fft_y = fft(img, axis=0)
# power_y = np.abs(fft_y)**2
# mean_power_y = power_y.mean(axis=1)
# freqs_y = fftshift(fftfreq(h, d=pixel_size_um))

# # Only use positive frequencies
# pos_mask = freqs_y > 0
# peaks, _ = find_peaks(mean_power_y[pos_mask], height=100, distance=5)

# 1. FFT along vertical axis
fft_y = fft(img, axis=0)
power_y = np.abs(fft_y)**2

# 2. Mean across columns and shift
mean_power_y = fftshift(power_y.mean(axis=1))
freqs_y = fftshift(fftfreq(h, d=pixel_size_um))

# 3. Use only positive frequencies
pos_mask = freqs_y > 0
pos_freqs = freqs_y[pos_mask]
pos_power = mean_power_y[pos_mask]

# 4. Find peaks on the aligned data
peaks, _ = find_peaks(pos_power, distance=5, prominence=0.1)

if len(peaks) > 0:
    first_peak_freq = freqs_y[pos_mask][peaks[0]]
    spacing_um_fft = 1 / first_peak_freq

    results = peak_widths(mean_power_y[pos_mask], peaks, rel_height=0.5)
    width_freq = results[0][0]
    spacing_um_fft_err = spacing_um_fft * (width_freq / peaks[0])
else:
    spacing_um_fft = spacing_um_fft_err = None

# === AUTOCORRELATION ANALYSIS ===
power_spec = np.abs(f)**2
autocorr = fftshift(ifft2(power_spec).real)
autocorr /= autocorr.max()

# Analyze Y-direction autocorrelation
autocorr_y = autocorr.mean(axis=1)
lags = np.arange(-h//2, h//2) * pixel_size_um
center_idx = len(lags) // 2
lags_positive = lags[center_idx:]
autocorr_y_positive = autocorr_y[center_idx:]

peaks_auto, _ = find_peaks(autocorr_y_positive, height=0.1, distance=5)
if len(peaks_auto) > 0:
    spacing_um_auto = lags_positive[peaks_auto[0]]
    results_auto = peak_widths(autocorr_y_positive, peaks_auto, rel_height=0.5)
    width_lag = results_auto[0][0]
    spacing_um_auto_err = spacing_um_auto * (width_lag / peaks_auto[0])
else:
    spacing_um_auto = spacing_um_auto_err = None

# === PLOTTING ===
plt.figure(figsize=(14, 4))
plt.subplot(1, 3, 1)
plt.imshow(np.log1p(np.abs(fshift)), cmap='gray')
plt.title("FFT Spectrum (log scale)")

plt.subplot(1, 3, 2)
# plt.plot(freqs_y, fftshift(mean_power_y))
# plt.plot(freqs_y[pos_mask][peaks], mean_power_y[pos_mask][peaks], "rx")
plt.plot(pos_freqs, pos_power)
plt.plot(pos_freqs[peaks], pos_power[peaks], "rx")
plt.title("Mean Power Spectrum (Y direction)")
plt.xlabel("Spatial Frequency (1/µm)")
plt.ylabel("Power")

plt.subplot(1, 3, 3)
plt.plot(lags_positive, autocorr_y_positive)
plt.plot(lags_positive[peaks_auto], autocorr_y_positive[peaks_auto], "rx")
plt.title("Autocorrelation Profile (Y direction)")
plt.xlabel("Lag (µm)")
plt.ylabel("Correlation")

plt.tight_layout()
plt.show()

# === RESULTS ===
print("\n=== ESTIMATED SPACING ===")
if spacing_um_fft:
    print(f"FFT Estimate: {spacing_um_fft:.2f} µm ± {spacing_um_fft_err:.2f} µm (FWHM)")
else:
    print("FFT Estimate: No clear peak found.")

if spacing_um_auto:
    print(f"Autocorrelation Estimate: {spacing_um_auto:.2f} µm ± {spacing_um_auto_err:.2f} µm (FWHM)")
else:
    print("Autocorrelation Estimate: No clear peak found.")


In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.ndimage import fourier_shift

# # === SETTINGS ===

# padding_factor = 1.1   # image will be padded to this times its size

# # === LOAD AND PAD IMAGE ===
# img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE).astype(float)
# h, w = img.shape
# pad_h = int(h * padding_factor)
# pad_w = int(w * padding_factor)

# # Pad to center image
# pad_img = np.zeros((pad_h, pad_w))
# start_h = (pad_h - h) // 2
# start_w = (pad_w - w) // 2
# pad_img[start_h:start_h+h, start_w:start_w+w] = img

# # === COMPUTE 2D FFT ===
# f = np.fft.fft2(pad_img)
# fshift = np.fft.fftshift(f)
# magnitude = np.abs(fshift)
# magnitude_log = np.log(1 + magnitude)

# # === COMPUTE 2D AUTOCORRELATION ===
# power_spectrum = np.abs(f) ** 2
# autocorr = np.fft.ifft2(power_spectrum).real
# autocorr = np.fft.fftshift(autocorr)
# autocorr /= np.max(autocorr)

# # === PLOT RESULTS ===
# fig, axs = plt.subplots(1, 3, figsize=(18, 6))

# axs[0].imshow(pad_img, cmap='gray')
# axs[0].set_title("Padded Input Image")

# axs[1].imshow(magnitude_log, cmap='gray')
# axs[1].set_title("FFT Spectrum (log scale)")

# axs[2].imshow(autocorr, cmap='hot', extent=[
#     -(pad_w // 2) * pixel_size_um,
#     (pad_w // 2) * pixel_size_um,
#     -(pad_h // 2) * pixel_size_um,
#     (pad_h // 2) * pixel_size_um,
# ])
# axs[2].set_title("2D Autocorrelation (centered)")
# axs[2].set_xlabel("X Lag (µm)")
# axs[2].set_ylabel("Y Lag (µm)")

# plt.tight_layout()
# plt.show()


In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from scipy.fft import fft2, fftshift, ifft2
# from scipy.signal import find_peaks

# image_path = "W:\\Users\\Daphne\\Imaging_Daphne\\25-06-02_XC_ptetwt_ExM\\MAX_020625_ExM_ptetWT_20xw_NSpark_288x0.589_002-ventral-crop-2rotated.png"
# pixel_size_um = 0.287

# img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE).astype(float)
# f = fft2(img)
# power_spectrum = np.abs(f) ** 2
# autocorr = fftshift(ifft2(power_spectrum).real)
# autocorr /= np.max(autocorr)

# # --- Radial profile of autocorrelation ---
# center = np.array(autocorr.shape) // 2
# Y, X = np.indices(autocorr.shape)
# r = np.hypot(X - center[1], Y - center[0])
# r_int = r.astype(int)
# radial_profile = np.bincount(r_int.ravel(), autocorr.ravel()) / np.bincount(r_int.ravel())

# # Convert pixel distances to micrometers
# r_um = np.arange(len(radial_profile)) * pixel_size_um

# # --- Find peaks in the radial profile ---
# peaks, _ = find_peaks(radial_profile, height=0.53, distance=10)

# plt.figure(figsize=(8, 5))
# plt.plot(r_um, radial_profile)
# plt.plot(r_um[peaks], radial_profile[peaks], "rx", label="Peaks")
# plt.title("Radial Line Profile of 2D Autocorrelation")
# plt.xlabel("Distance from Center (µm)")
# plt.ylabel("Mean Autocorrelation")
# plt.grid(True)
# plt.legend()
# plt.show()

# # Print peak distances
# print("Peak distances (µm):", r_um[peaks])